In [13]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import copy

# from mlflow import log_metric, log_param, log_artifacts
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

from mlmvn.layers import FirstLayer, HiddenLayer, OutputLayer, cmplx_phase_activation
from mlmvn.loss import ComplexMSELoss

In [10]:
def reverse_one_hot(x, neuronCats):
    a = np.zeros(len(x))
    x = torch.detach(x)
    for i in range(len(x)):
        a[i] = torch.max(x[i]) - 1 + np.argmax(x[i]) * neuronCats
    return a

def accuracy(out, yb):
    out = out.type(torch.double)
    yb = yb.type(torch.double)
    x = 0
    for i in range(len(out)):
        x += torch.equal(out[i], yb[i])
    return x / len(out)

In [9]:
# ------- control variables: -------
# number of categories a neuron can distinguish / parameter that determines the number of output neurons
neuronCats = 1
# number of categories per neuron, i.e. neuronCats (+ 1 for others in case of multiple Outputs)
categories = 2
# how often a classification sector occurs (1 means no periodicity)
periodicity = 1

## Load Data

In [3]:
train_csv = pd.read_csv(
    "autass_data2.csv",
    header=None,
    dtype=np.double,
)
data = np.array(train_csv.values[:, 1:50])
del train_csv

In [16]:
X = data[:, 0:48]
y = data[:, 48].astype(int) - 1

yt = copy.copy(y)
yt[yt == 0] = 20
yt[yt == 1] = 21
yt[yt == 2] = 22
yt[yt == 3] = 23
yt[yt == 4] = 26
yt[yt == 5] = 24
yt[yt == 6] = 27
yt[yt == 7] = 29
yt[yt == 8] = 30
yt[yt == 9] = 25
yt[yt == 10] = 28
yt -= 20
y = yt
del yt

In [17]:
x_train, x_valid, y_train, y_valid = train_test_split(
        X, y, train_size=46806, random_state=42
    )

# one-hot encoding
numSamples, numFeatures = x_valid.shape
y_valid_int = y_valid
y2 = y_valid + 1  # auxiliary variable so that classes start at 1 and not 0
numClasses = max(y2)
target_ids = range(numClasses)
no = int(np.ceil(numClasses / neuronCats))  # number of output neurons
if no != 1:
    y_valid = torch.zeros(numSamples, no)
    for i in range(numSamples):
        k = int(np.ceil(y2[i] / neuronCats)) - 1
        c = np.mod((y2[i] - 1), neuronCats) + 1
        y_valid[i, k] = c
numSamples, numFeatures = x_train.shape
y_train_int = y_train
y2 = y_train + 1  # auxiliary variable so that classes start at 1 and not 0
if no != 1:
    y_train = torch.zeros(numSamples, no)
    for i in range(numSamples):
        k = int(np.ceil(y2[i] / neuronCats)) - 1
        c = np.mod((y2[i] - 1), neuronCats) + 1
        y_train[i, k] = c
del y2

# Convert numpy arrays into torch tensors
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
if y_train.size().__len__() == 1:
    y_train = torch.unsqueeze(y_train, 1)
    y_valid = torch.unsqueeze(y_valid, 1)

# convert angles to complex numbers on unit-circle
x_train = torch.exp(1.0j * x_train)
x_valid = torch.exp(1.0j * x_valid)

# ns = number of samples , ni = number of inputs / input neurons
ns, ni = x_train.shape

<ipython-input-17-67ce26329374>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train, y_train, x_valid, y_valid = map(


## Model

In [15]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act2 = cmplx_phase_activation()

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def predict(self, x, loss):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return output
